<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosI/blob/master/notebooks/3c_minhash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda de documentos con MinHash
En esta libreta veremos cómo hacer búsqueda eficiente de documentos considerando la similitud de Jaccard.

La similitud de Jaccard entre un par de conjuntos $(\mathcal{C}^{(1)}, \mathcal{C}^{(2)})$ está dada por

$$
J(\mathcal{C}^{(1)}, \mathcal{C}^{(2)}) = \frac{\mid \mathcal{C}^{(1)} \cap \mathcal{C}^{(2)} \mid}{\mid \mathcal{C}^{(1)}\cup \mathcal{C}^{(2)} \mid} \in [0,1]
$$

In [1]:
from collections import Counter
from math import floor, log
import codecs 
import re 

import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

VOCMAX = 5000
n_muestras_sint = 10000
n_tablas_ng = 50

# Para reproducibilidad
np.random.seed(2021)

## Conjuntos de datos
Preparamos dos conjuntos de datos para probar los algoritmos de búsqueda.

### Datos sintéticos
Este conjunto de datos está compuesto por 5 listas de elementos y un conjunto universo de 10 elementos.


In [2]:
sint_conj = [[0, 1, 4, 6, 8], [2, 3, 4, 7, 8], [1, 4, 6, 7], [0, 5, 6, 8], [0, 1, 3, 4, 7]]
univ = {e for l in sint_conj for e in l}

Calculamos la similitud de Jaccard de todos los pares.

In [3]:
sims_jacc = np.identity(len(sint_conj))
for i in range(0, len(sint_conj) - 1):
  ci = set(sint_conj[i])
  for j in range(i + 1, len(sint_conj)):
    cj = set(sint_conj[j])
    sims_jacc[i,j] = float(len(ci.intersection(cj)) / len(ci.union(cj)))
    sims_jacc[j,i] = sims_jacc[i,j]

print(sims_jacc)

[[1.         0.25       0.5        0.5        0.42857143]
 [0.25       1.         0.28571429 0.125      0.42857143]
 [0.5        0.28571429 1.         0.14285714 0.5       ]
 [0.5        0.125      0.14285714 1.         0.125     ]
 [0.42857143 0.42857143 0.5        0.125      1.        ]]


Definimos una función que calcula la propiedad de colisión de todos los pares en este conjunto de datos

In [4]:
def p_colision(db, tablas, n_tablas, n_ej):
  for i in range(n_tablas):
    for j,l in enumerate(db):
      tablas[i].insertar(l, j)

  colisiones = np.zeros((n_ej, n_ej))
  for i in range(n_tablas):
    for j,cj in enumerate(db):
      for e in tablas[i].buscar(cj):
        colisiones[j, e] += 1

  return colisiones / n_tablas

### 20 Newsgroups
Vamos a usar el conjunto de documentos de _20 Newsgropus_, el cual descargamos usando scikit-learn. 

In [5]:
db = fetch_20newsgroups(remove=('headers','footers','quotes'))

Importamos la biblioteca NLTK y definimos nuestro analizador léxico y lematizador

In [6]:
import nltk
nltk.download(['punkt','averaged_perceptron_tagger','wordnet'])

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.corpus.reader.wordnet import NOUN, VERB, ADV, ADJ

morphy_tag = {
    'JJ' : ADJ,
    'JJR' : ADJ,
    'JJS' : ADJ,
    'VB' : VERB,
    'VBD' : VERB,
    'VBG' : VERB,
    'VBN' : VERB,
    'VBP' : VERB,
    'VBZ' : VERB,
    'RB' : ADV,
    'RBR' : ADV,
    'RBS' : ADV
}

def doc_a_tokens(doc):
  tagged = pos_tag(word_tokenize(doc.lower()))
  lemmatizer = WordNetLemmatizer()
  tokens = []
  for p,t in tagged:
    tokens.append(lemmatizer.lemmatize(p, pos=morphy_tag.get(t, NOUN)))

  return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Convertimos el conjunto preprocesado a una lista de cadenas, una por documento

In [7]:
corpus = []
for d in db.data:
  d = d.replace('\n',' ').replace('\r',' ').replace('\t',' ')
  d = ' '.join([''.join([c.lower() for c in p if c.isalnum()]) for p in d.split()])
  tokens = doc_a_tokens(d)
  corpus.append(' '.join(tokens))

Dividimos nuestro conjunto en 2 subconjuntos: los documentos de la base que se buscarán y los documentos de consulta

In [8]:
perm = np.random.permutation(len(corpus)).astype(int)
n_ej_base = int(floor(len(corpus) * 0.95))

base = [corpus[i] for i in perm[:n_ej_base]]
consultas = [corpus[i] for i in perm[n_ej_base:]]

Obtenemos y cargamos la lista de _stopwords_ para inglés (archivo con una palabra por línea)

In [9]:
!wget -qO- -O stopwords_english.txt \
         https://raw.githubusercontent.com/pan-webis-de/authorid/master/data/stopwords_english.txt

stopwords = []
for line in codecs.open('stopwords_english.txt', encoding = "utf-8"):
  stopwords.append(line.rstrip())

Procesamos cada palabra del corpus completo para generar y ordenar el vocabulario

In [10]:
# Divide la cadena en palabras
term_re = re.compile("\w+", re.UNICODE)

# Contamos las ocurrencias de cada palabra
corpus_freq = Counter()
doc_freq = Counter()
for d in base:
  # Eliminamos números de la cadena (documento) a procesar 
  d = re.sub(r'\d+', '', d)

  # Dividimos la cadena en una lista de palabras
  terms = [t for t in term_re.findall(d) if t not in stopwords and len(t) > 2]
  
  # Aumentamos el contador de cada instancia palabra en el documento
  for t in terms:
    corpus_freq[t] += 1
  
  # Aumentamos el contador de cada palabra distinta en el documento
  for t in set(terms):
    doc_freq[t] += 1

# Generamos un diccionario con las VOCMAX palabras más frecuentes
vocabulary = {entry[0]:(i, entry[1], doc_freq[entry[0]], log(len(corpus) / doc_freq[entry[0]])) \
              for i, entry in enumerate(corpus_freq.most_common()) \
              if i < VOCMAX}

Creamos un diccionario para mapear índices a palabras

In [11]:
id_a_palabra = {v[0]: k for k,v in vocabulary.items()}

Generamos las bolsas de palabras de los documentos preprocesados

In [12]:
def cadenas_a_bolsas(cadenas, voc, descartar, tre):
  bolsas = []
  for c in cadenas:
    c = re.sub(r'\d+', '', c)
    ids = Counter([voc[t][0] for t in tre.findall(c) \
                   if t in voc and t not in descartar])
    bolsas.append([i for i in sorted(ids.items())])

  return bolsas

bolsas_base = cadenas_a_bolsas(base, vocabulary, stopwords, term_re)
bolsas_consultas = cadenas_a_bolsas(consultas, vocabulary, stopwords, term_re)

In [13]:
def csr_a_ldb(csr):
  ldb = [[] for _ in range(csr.shape[0])]
  coo = csr.tocoo()    
  for i,j,v in zip(coo.row, coo.col, coo.data):
    ldb[i].append(j)

  return ldb

def ldb_a_csr(ldb, dim):
  n_el = 0
  for i,l in enumerate(ldb):
    n_el += len(l)

  vals = np.zeros(n_el, dtype=float)
  rows = np.zeros(n_el, dtype=int)
  cols = np.zeros(n_el, dtype=int)
  j = 0
  for i,l in enumerate(ldb):
    for e in l:
      vals[j] = e[1]
      rows[j] = i
      cols[j] = e[0]
      j += 1
  return csr_matrix((vals, (rows, cols)), shape=(len(ldb), dim))

bolsas_base_csr = ldb_a_csr(bolsas_base, VOCMAX)
bolsas_consultas_csr = ldb_a_csr(bolsas_consultas, VOCMAX)

## MinHash binario
Min-Hashing es un algoritmo para búsqueda de conjuntos similares bajo la similitud de Jaccard, la cual consiste en:
* Generar permutación aleatoria del conjunto universo $\mathbb{U}$
* Asignar a cada conjunto su 1er elemento bajo la permutación, esto es,
$$
h(\mathcal{C}^{(i)}) = min(\pi(\mathcal{C}^{(i)}))
$$

La probabilidad de que dos conjuntos tengan valor MinHash idéndico es igual a su similitud de Jaccard:
$$
P[h(\mathcal{C}^{(i)}) = h(\mathcal{C}^{(j)})] = \frac{\mid \mathcal{C}^{(i)} \cap \mathcal{C}^{(j)} \mid}{\mid \mathcal{C}^{(i)}\cup \mathcal{C}^{(j)} \mid} \in [0,1]
$$

Para buscar conjuntos similares, los valores MinHash se agrupan en $l$ tuplas de
$r$ funciones distintas de la siguiente forma:    

\begin{align*}
 g_1(\mathcal{C}^{(i)}) & = (h_1(\mathcal{C}^{(i)}), h_2(\mathcal{C}^{(i)}), \ldots , h_r(\mathcal{C}^{(i)}))\\
g_2(\mathcal{C}^{(i)}) & = (h_{r+1}(\mathcal{C}^{(i)}), h_{r+2}(\mathcal{C}^{(i)}), \ldots , h_{2\cdot r}(\mathcal{C}^{(i)}))\\
      \cdots\\
g_l(\mathcal{C}^{(i)}) & = (h_{(l-1)\cdot r+1}(\mathcal{C}^{(i)}), h_{(l-1)\cdot r2}(\mathcal{C}^{(i)}), \ldots , h_{l\cdot r}(\mathcal{C}^{(i)}))
\end{align*}
    
Conjuntos con una tupla idéntica se almacenan en la misma cubeta en la tabla asociada a la tupla.

In [14]:
class MinHashTable:
  def __init__(self, n_cubetas, t_tupla, dim):
    self.n_cubetas = n_cubetas
    self.tabla = [[] for i in range(n_cubetas)]
    self.dim = dim
    self.t_tupla = t_tupla
    
    self.perm = np.random.uniform(0, 1, size=(self.t_tupla, self.dim))
    self.rind = np.random.randint(0, np.iinfo(np.int32).max, size=(self.t_tupla, self.dim))

    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291

  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo

  def minhash(self, x):
    xp = self.perm[:, x]
    xi = self.rind[:, x]
    amin = xp.argmin(axis = 1)
    emin = xi[:, amin]

    return np.sum(self.a * emin, dtype=np.ulonglong), np.sum(self.b * emin, dtype=np.ulonglong)
     
  def insertar(self, x, ident):
    mh, v2 = self.minhash(x)
  
    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(mh)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == mh:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1]
        
    return []

  def eliminar(self, x, ident):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        break
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1].remove(ident)

    return -1

### Verificación con conjunto de datos sintéticos
Primero verificamos la probabilidad de colisión de dos conjuntos en nuestra implementación.

In [15]:
tablas_sint_bin = [MinHashTable(2**4, 1, len(univ)) for _ in range(n_muestras_sint)]
print(p_colision(sint_conj, tablas_sint_bin, n_muestras_sint, len(sint_conj)))

[[1.     0.2471 0.5046 0.5002 0.4253]
 [0.2471 1.     0.2827 0.1235 0.4242]
 [0.5046 0.2827 1.     0.1477 0.5002]
 [0.5002 0.1235 0.1477 1.     0.1224]
 [0.4253 0.4242 0.5002 0.1224 1.    ]]


### Búsqueda de documentos similares con _20 newsgroups_
Probamos la implementación de Min-Hashing para la búsqueda de documentos similares en _20 newsgroups_. Para realizar la búsqueda de documentos similares:

1. Insertamos las listas a nuestras tablas
2. Recuperamos los documentos similares a nuestros documentos de consulta usando las tablas MinHash.
3. Calculamos la similitud Jaccard de los documentos recuperados con los de consulta 
4. Ordenamos por similitud.

In [16]:
def similitud_jaccard(x, y):
  x = x.toarray()[0]
  y = y.toarray()[0]
  inter = np.count_nonzero(x * y)
  return inter / (np.count_nonzero(x) + np.count_nonzero(y) - inter)


def similitud_minmax(x, y):
  x = x.toarray()[0]
  y = y.toarray()[0]  
  min = np.min(np.array([x, y]).T, axis=1).sum()
  max = np.max(np.array([x, y]).T, axis=1).sum()
  return min / max

def similitud_minmax_pesado(x, y, w):
  x = x.toarray()[0]
  y = y.toarray()[0]
  min = (w * np.min(np.array([x, y]).T, axis=1)).sum()
  max = (w * np.max(np.array([x, y]).T, axis=1)).sum()
  return min / max
  
def fuerza_bruta(ds, qs, fs):
  medidas = np.zeros(ds.shape[0])
  for i,x in enumerate(ds):
    medidas[i] = fs(qs, x)
  return np.sort(medidas)[::-1], np.argsort(medidas)[::-1]

def busca_pares_documentos(base_csr, 
                           consultas_csr, 
                           base_ldb, 
                           consultas_ldb, 
                           tablas, 
                           fs):
  for j,l in enumerate(base_ldb):
    for i in range(len(tablas)):
      if l:
        tablas[i].insertar(l, j)

  docs = []
  for j,l in enumerate(consultas_ldb):
    dc = []
    if l:
      for i in range(len(tablas)):
        dc.extend(tablas[i].buscar(l))
    docs.append(set(dc))

  sims = []
  orden = []
  for i,q in enumerate(consultas_csr):
    ld = list(docs[i])
    if ld:
      s,o = fuerza_bruta(base_csr[ld], q, fs)
      sims.append(s)
      orden.append([ld[e] for e in o])
    else:
      sims.append([])
      orden.append([])

  return sims, orden

Buscamos documentos similares y examinamos un ejemplo de consulta y su correspondiente documento más similar.

In [17]:
tablas_ng_bin = [MinHashTable(2**18, 2, VOCMAX) for _ in range(n_tablas_ng)]
sims, orden = busca_pares_documentos(bolsas_base_csr, 
                                     bolsas_consultas_csr,
                                     bolsas_base, 
                                     bolsas_consultas, 
                                     tablas_ng_bin, 
                                     similitud_jaccard)
print("------ C O N S U L T A ------\n", consultas[21])
print("\n------ M Á S  S I M I L A R ------\n", base[list(orden[21])[0]])

------ C O N S U L T A ------
 actually the book be call seventh day adventist believe and there be 27 basica belief i believe it be print by the reveiew and herald publishing association competition be the law of the jungle cooperation be the law of civilization eldridge cleaver

------ M Á S  S I M I L A R ------
 do the word chill effect stimulate impulse within that small collection of neuron you call a brain cpk it be 80 day do you know where your wallet be


## MinHashing con multiplicidades enteras
Si tenemos bolsas con multiplicidades enteras bajo la similitud MinMax, podemos convertir cada bolsa $\mathcal{B}^{(i)}$ a un conjunto $\hat{\mathcal{C}}^{(i)}$, reemplazando cada multiplicidad con un elemento distinto. El conjunto universal extentido sería
\begin{equation*}
  U_{ext} = \{1, \ldots, F_1, \ldots, F_1 + \cdots + F_{D - 1} + 1, \ldots , F_1 + \cdots + F_D \}
\end{equation*}

donde $F_1, \ldots , F_D$ son las multiplicidades máximas de los elementos $1, \ldots, D$ 

Si aplicamos el esquema de Min-Hashing descrito anteriormente a los conjuntos $\hat{\mathcal{C}}^{(i)}, \hat{\mathcal{C}}^{(j)} \subseteq U_{ext}$ se cumple que

$$
P[h(\hat{\mathcal{C}}^{(i)}) = h(\hat{\mathcal{C}}^{(j)})] = \frac{\sum_{k = 1}^{d} \min (\mathcal{B}^{(i)}_k, \mathcal{B}^{(j)}_k)}{\sum_{k = 1}^{d} \max(\mathcal{B}^{(i)}_k, \mathcal{B}^{(j)}_k)} = J_{\mathcal{B}}(\mathcal{B}^{(i)}, \mathcal{B}^{(j)}) 
$$ 

In [18]:
class ExtenderBolsas:
  def fit(self, bolsas, pesos=None):
    maxfrecs = bolsas.max(axis=0).toarray()[0]
    self.cumsums = np.zeros(maxfrecs.size, dtype=np.int32)
    self.cumsums[1:] = np.cumsum(maxfrecs[:-1])
    self.t_voc = maxfrecs.sum()

  def transform(self, bolsas):
    bolsas_ext = lil_matrix((bolsas.shape[0], int(self.t_voc)), dtype=np.int32)
    b = bolsas.tocoo()
    for i,j,v in zip(b.row, b.col, b.data):
      bolsas_ext[i, int(self.cumsums[j]):int(self.cumsums[j]+v)] = 1
    
    return bolsas_ext.tocsr()

  def fit_transform(self, bolsas):
    self.fit(bolsas)
    return self.transform(bolsas)

  def transform_weights(self, w):
    w_ext = np.zeros(int(self.t_voc))
    for i in range(w.size - 1):
      w_ext[int(self.cumsums[i]):int(self.cumsums[i+1])] = w[i]
    w_ext[int(self.cumsums[-1]):] = w[-1]

    return w_ext

### Verificación con datos sintéticos
Generamos bolsas con multiplicidades enteras.

In [19]:
filas = [i for i,l in enumerate(sint_conj) for _ in range(len(l))]
cols = [e for l in sint_conj for e in l]
vals = [np.random.randint(1, 5) for l in sint_conj for e in l]
sint_csr = csr_matrix((vals, (filas, cols)), shape=(len(sint_conj), len(univ)))
eb_sint = ExtenderBolsas()
sint_ext = eb_sint.fit_transform(sint_csr)

Calculamos la similitud MinMax de todos los pares. Esta similitud está dada por


$$
J_{\mathcal{B}}(\mathcal{B}^{(i)}, \mathcal{B}^{(j)})  = \frac{\sum_{k = 1}^{d} \min (\mathcal{B}^{(i)}_k, \mathcal{B}^{(j)}_k)}{\sum_{k = 1}^{d} \max(\mathcal{B}^{(i)}_k, \mathcal{B}^{(j)}_k)}
$$ 

In [20]:
sims_minmax = np.identity(sint_csr.shape[0])
for i in range(0, sint_csr.shape[0] - 1):
  bi = sint_csr[i]
  for j in range(i + 1, sint_csr.shape[0]):
    bj = sint_csr[j]
    sims_minmax[i,j] = similitud_minmax(bi, bj)
    sims_minmax[j,i] = sims_minmax[i,j]
print(sims_minmax)

[[1.         0.26086957 0.5625     0.3        0.27777778]
 [0.26086957 1.         0.2        0.19047619 0.22222222]
 [0.5625     0.2        1.         0.10526316 0.28571429]
 [0.3        0.19047619 0.10526316 1.         0.11764706]
 [0.27777778 0.22222222 0.28571429 0.11764706 1.        ]]


Verificamos la proporción de colisiones entre cada par de bolsas y la comparamos con las similitudes calculadas.

In [21]:
tablas_mult = [MinHashTable(2**4, 1, sint_ext.shape[-1]) for _ in range(n_muestras_sint)]
print(p_colision(csr_a_ldb(sint_ext), tablas_mult, n_muestras_sint, len(sint_conj)))

[[1.     0.268  0.5598 0.302  0.2776]
 [0.268  1.     0.2018 0.1957 0.2238]
 [0.5598 0.2018 1.     0.1025 0.2765]
 [0.302  0.1957 0.1025 1.     0.1187]
 [0.2776 0.2238 0.2765 0.1187 1.    ]]


### Búsqueda de documentos similares con _20 newsgroups_
Probamos la implementación buscando documentos similares en _20 newsgroups_ bajo la similitud MinMax y examinamos un ejemplo de consulta y su correspondiente documento más similar.

In [22]:
eb_ng = ExtenderBolsas()
bolsas_base_ext = eb_ng.fit_transform(bolsas_base_csr)
bolsas_consultas_ext = eb_ng.transform(bolsas_consultas_csr)
tablas_ng_multent = [MinHashTable(2**18, 2, bolsas_base_ext.shape[-1]) for _ in range(n_tablas_ng)]
sims, orden = busca_pares_documentos(bolsas_base_ext, 
                                     bolsas_consultas_ext, 
                                     csr_a_ldb(bolsas_base_ext), 
                                     csr_a_ldb(bolsas_consultas_ext), 
                                     tablas_ng_multent, 
                                     similitud_minmax)
print("------ C O N S U L T A ------\n", consultas[21])
print("\n------ M Á S  S I M I L A R ------\n", base[list(orden[21])[0]])

------ C O N S U L T A ------
 actually the book be call seventh day adventist believe and there be 27 basica belief i believe it be print by the reveiew and herald publishing association competition be the law of the jungle cooperation be the law of civilization eldridge cleaver

------ M Á S  S I M I L A R ------
 88 toyota camry top of the line vehicle blue book 10500 ask 9900 73 k mile auto transmission have everything own by a meticulous automoble mechanic call 408 4258203 ask for bob


### MinHashing con pesos asociados a elementos
También podemos incluir pesos para cada elemento del conjunto universo y calcular una similitud MinMax que tome en cuenta estos pesos como:

$$
J_{\mathcal{B}_{p}}(\mathcal{B}^{(i)}, \mathcal{B}^{(j)}, \mathbf{w}) = \frac{\sum_{k = 1}^{d} w_k \cdot \min (\mathcal{B}^{(i)}_k, \mathcal{B}^{(j)}_k)}{\sum_{k = 1}^{d} w_k \cdot \max(\mathcal{B}^{(i)}_k, \mathcal{B}^{(j)}_k)} = J_{\mathcal{B}}(\mathcal{B}^{(i)}_k, \mathcal{B}^{(j)}_k) 
$$ 

Una ejemplo de pesado de elementos es el _inverse document frecuency_, el cual se obtiene aplicando el logaritmo a la división del número total de documentos $n$ en la colección entre el número de documentos en los que ocurre la palabra. 

Podemos extender el esquema de Min-Hashing definido anteriormente para que tome en cuenta pesos sobre los elementos. Esto se logra transformando el valor aleatorio asociado a un elemento como sigue: 
$$
\hat{x}_k = \frac{-\log{x_k}}{w_k}, x_k \sim Unif(0,1)
$$

donde $x_k$ es el número asociado al elemento $k$, $w_k$ es el peso del mismo y $\hat{x}_k$ es el valor transformado.

In [23]:
class MinHashPesadoTable:
  def __init__(self, n_cubetas, t_tupla, dim, weights):
    self.n_cubetas = n_cubetas
    self.tabla = [[] for i in range(n_cubetas)]
    self.dim = dim
    self.t_tupla = t_tupla
    
    self.perm = np.random.uniform(1, 0, size=(self.t_tupla, self.dim))
    for i in range(self.t_tupla):
      for j in range(self.dim):
        self.perm[i][j] = -np.log(self.perm[i][j]) / weights[j]

    self.rind = np.random.randint(0, np.iinfo(np.int32).max, size=(self.t_tupla, self.dim))
    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291    
      
  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo

  def minhash(self, x):
    xp = self.perm[:, x]
    xi = self.rind[:, x]
    amin = xp.argmin(axis = 1)
    emin = xi[:, amin]

    return np.sum(self.a * emin, dtype=np.ulonglong), np.sum(self.b * emin, dtype=np.ulonglong)
     
  def insertar(self, x, ident):
    mh, v2 = self.minhash(x)
  
    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(mh)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == mh:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1]
        
    return []

  def eliminar(self, x, ident):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        break
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1].remove(ident)

    return -1

### Verificación con datos sintéticos
Asignamos pesos de forma aleatoria a cada elemento y calculamos la similitud MinMax pesada para todos los pares.

In [24]:
w = np.random.uniform(0, 7, size=sint_csr.shape[-1])
sims_minmax_pesado = np.identity(sint_csr.shape[0])
for i in range(0, sint_csr.shape[0] - 1):
  bi = sint_csr[i]
  for j in range(i + 1, sint_csr.shape[0]):
    bj = sint_csr[j]
    sims_minmax_pesado[i,j] = similitud_minmax_pesado(bi, bj, w)
    sims_minmax_pesado[j,i] = sims_minmax_pesado[i,j]
print(sims_minmax_pesado)

[[1.         0.26753287 0.63996502 0.27834711 0.303941  ]
 [0.26753287 1.         0.21114275 0.17702148 0.23387911]
 [0.63996502 0.21114275 1.         0.13539084 0.2585339 ]
 [0.27834711 0.17702148 0.13539084 1.         0.11604851]
 [0.303941   0.23387911 0.2585339  0.11604851 1.        ]]


Verificamos la proporción de colisiones entre cada par y lo comparamos con las similitudes calculadas.

In [25]:
w_ext = eb_sint.transform_weights(w)
tablas_mult_p = [MinHashPesadoTable(2**4, 1, sint_ext.shape[-1], weights=w_ext) for _ in range(n_muestras_sint)]
print(p_colision(csr_a_ldb(sint_ext), tablas_mult_p, n_muestras_sint, len(sint_conj)))

[[1.     0.2727 0.6363 0.2781 0.3137]
 [0.2727 1.     0.2159 0.175  0.231 ]
 [0.6363 0.2159 1.     0.1329 0.2618]
 [0.2781 0.175  0.1329 1.     0.1149]
 [0.3137 0.231  0.2618 0.1149 1.    ]]


### Búsqueda de documentos similares con _20 newsgroups_
Calculamos el _inverse document frecuency_ (IDF) de cada palabra del conjunto de _20 newsgroups_.

In [26]:
idf = np.zeros(bolsas_base_csr.shape[-1])
filas, cols = bolsas_base_csr.nonzero()
for i in range(bolsas_base_csr.shape[-1]):
  idf[i] = np.log(bolsas_base_csr.shape[0] / (i == cols).sum())
idf = eb_ng.transform_weights(idf)

Realizamos la búsqueda usando MinHash con pesado IDF para las palabras.

In [28]:
tablas_ng_pesado = [MinHashPesadoTable(2**18, 2, bolsas_base_ext.shape[-1], weights=idf) for _ in range(n_tablas_ng)]
sims, orden = busca_pares_documentos(bolsas_base_ext, 
                                     bolsas_consultas_ext, 
                                     csr_a_ldb(bolsas_base_ext), 
                                     csr_a_ldb(bolsas_consultas_ext), 
                                     tablas_ng_pesado, 
                                     similitud_minmax)
print("------ C O N S U L T A ------\n", consultas[21])
print("\n------ M Á S  S I M I L A R ------\n", base[list(orden[21])[0]])

------ C O N S U L T A ------
 actually the book be call seventh day adventist believe and there be 27 basica belief i believe it be print by the reveiew and herald publishing association competition be the law of the jungle cooperation be the law of civilization eldridge cleaver

------ M Á S  S I M I L A R ------
 maybe im too religious but when i see a bill to establish a right i wince keep in mind what the law giveth the law can taketh away


## Ejercicio
+ Prueba con otros hiperparámetros.